In [87]:
import pandas as pd

df = pd.read_csv('full_set.csv')
df = df.dropna()
cols = list(df.columns)
#remove date colums
cols = cols[1:]

In [88]:
# Generate list of all possible pairs
# Using list comprehension + enumerate() 

res = [(a, b) for idx, a in enumerate(cols) for b in cols[idx + 1:]] 

res[0:4]

[('CL1', 'CL2'), ('CL1', 'CL3'), ('CL1', 'CL4'), ('CL1', 'CL5')]

In [89]:
def calc_tc(spread):
    tcs = {'CO': 0.000289,
          'CL': 0.000940,
          'HO': 0.002482,
          'QS': 0.002482,
          'XB': 0.002482}
    return tcs[spread[0][0:2]] + tcs[spread[0][0:2]]

In [84]:
### function - implementaion of maximum expected return
### inputs:
### a - value for alpha
### n - value for eta
### c - transaction costs
### returns:
### res - rseult for a using eq. (18)


def mer(a,n,c):
    var1 = -c/4
    denom1 = (c**3*a**3) + (24*c*a**2*n**2)
    denom2 = 4*(np.sqrt(((3*c**4*a**5*n**2) + (36*c**2*a**4*n**4))))
    denom3 = 4 * ((denom1 - denom2)**(1/3))
    var2 = (c**2*a) / denom3
    numer1 = c**3*a**3 + (24*c*a**2*n**2)
    numer2 = 4 * np.sqrt((3*c**4*a**5*n**2 + 36*c**2*a**4*n**4))
    numer3 = ((numer1 - numer2))**(1/3)
    var3 = numer3 / (4*a)
    res = var1 - var2 - var3
    return res

In [92]:
import statsmodels.api as sm
import numpy as np
def OU_calibration(df, spread):
    dummy_df = pd.DataFrame()
    A = spread[0]
    B = spread[1]
    df['spread'] = np.log((df[A] / df[B]))
    df['shift'] = df.spread.shift()
    df.loc[0, 'shift'] = 0
    X = df['shift'].fillna(0)
    y = df['spread'].fillna(0)
    X = sm.add_constant(X)
    results = sm.OLS(y,X).fit()
    order = A, B
    
    if results.params[0] < 0:
        df['spread'] = np.log((df[B] / df[A]))
        df['shift'] = df.spread.shift()
        df.loc[0, 'shift'] = 0
        X = df['shift'].fillna(0)
        y = df['spread'].fillna(0)
        X = sm.add_constant(X)
        results = sm.OLS(y,X).fit()
        order = B, A
        
    
    a, b, sde= results.params[1], results.params[0], results.bse[1]
    
    alpha = -np.log(a)
    mu = b/(1-a)
    sigma =sde*np.sqrt((-2*np.log(a))/(1-a*a))
    
    return alpha, mu,sigma, order


In [99]:
results =pd.DataFrame()
for i in range(0,len(res)):
    spread = res[i]
    alpha, mu,sigma, order = OU_calibration(df, spread)
    c= calc_tc(order)
    a = mer(alpha,sigma,c)
    print (order, alpha, mu,sigma,c,a)
    results = results.append(
            {  'spread': order, 
               'alpha': alpha,
               'mu':mu,
               'sigma':sigma,
               't_c': c,
               'a':a}, ignore_index=True)




C:\Users\M052366\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


('CL2', 'CL1') 0.09212555018323036 0.01018512813087091 0.010090985461643396 0.00188 -0.012083622884545444
('CL3', 'CL1') 0.09543189799238541 0.017947542742892293 0.010270229808752886 0.00188 -0.012083442469380956
('CL4', 'CL1') 0.08881993853204824 0.023520769754632527 0.009907810159729307 0.00188 -0.012083248046853469
('CL5', 'CL1') 0.08141171159716624 0.02765306534805339 0.009485406006671611 0.00188 -0.012083071265239935
('CO1', 'CL1') 0.11994524930023813 0.0840251977841712 0.011475251368018356 0.000578 -0.00795467892811325
('CO2', 'CL1') 0.1283462598582724 0.08913266473424042 0.01187846711088625 0.000578 -0.007958245179144547
('CO3', 'CL1') 0.12316330353923408 0.0939936922017278 0.011639352559720413 0.000578 -0.007959683941839164
('CO4', 'CL1') 0.11570931499341779 0.09845297468622552 0.011283211727639545 0.000578 -0.007960407088091755
('CO5', 'CL1') 0.10833412880685389 0.1024248254204306 0.010918626895065658 0.000578 -0.007960844523747969
('HO1', 'CL1') 0.38020262285614925 1.17532219